## Numerical linear algebra: introduction

* Topics in numerical algebra: 
    - BLAS  
    - solve linear equations $\mathbf{A} \mathbf{x} = \mathbf{b}$
    - regression computations $\mathbf{X}^T \mathbf{X} \beta = \mathbf{X}^T \mathbf{y}$  
    - eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{x}$  
    - generalized eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{B} \mathbf{x}$  
    - singular value decompositions $\mathbf{A} = \mathbf{U} \Sigma \mathbf{V}^T$  
    - iterative methods  

* Except for the iterative methods, most of these numerical linear algebra tasks are implemented in the BLAS and LAPACK libraries. They form the **building blocks** of most statistical computing tasks (optimization, MCMC).

* Our major **goal** (or learning objectives) is to  
    0. know the computational cost (flop count) of each task
    0. be familiar with the BLAS and LAPACK functions (what they do)  
    0. do **not** re-invent wheels by implementing these dense linear algebra subroutines by yourself  
    0. understand the need for iterative methods  
    0. apply appropriate numerical algebra tools to various statistical problems 

* All high-level languages (R, Matlab, Julia) call BLAS and LAPACK for numerical linear algebra. 
    - Julia offers more flexibility by exposing interfaces to many BLAS/LAPACK subroutines directly. See [documentation](https://docs.julialang.org/en/stable/stdlib/linalg/#BLAS-Functions-1).

## BLAS

* BLAS stands for _basic linear algebra subprograms_. 

* See [netlib](http://www.netlib.org/blas/) for a complete list of standardized BLAS functions.

* There are many implementations of BLAS. 
    - [Netlib](http://www.netlib.org/blas/) provides a reference implementation  
    - Matlab uses Intel's [MKL](https://software.intel.com/en-us/node/520724) (mathematical kernel libaries)  
    - Julia uses [OpenBLAS](https://github.com/xianyi/OpenBLAS)  
    - JuliaPro offers the option of using MKL

* There are 3 levels of BLAS functions.
    - [Level 1](http://www.netlib.org/blas/#_level_1): vector-vector operation
    - [Level 2](http://www.netlib.org/blas/#_level_2): matrix-vector operation
    - [Level 3](http://www.netlib.org/blas/#_level_3): matrix-matrix operation

| Level | Example Operation                      | Name        | Dimension                                 | Flops |
|-------|----------------------------------------|-------------|-------------------------------------------|-------|
| 1     | $\alpha \gets \mathbf{x}^T \mathbf{y}$ | dot product | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ | $2n$  |
|       | $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$ |  axpy           |  $\alpha \in \mathbb{R}$, $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ |  $2n$    |
| 2     | $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ |  gaxpy           |  $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                     |  $2mn$     |
|       | $\mathbf{A} \gets \mathbf{A} + \mathbf{y} \mathbf{x}^T$ | rank one update            |    $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                       | $2mn$      |
| 3     | $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$                                       |  matrix multiplication           |  $\mathbf{A} \in \mathbb{R}^{m \times p}$, $\mathbf{B} \in \mathbb{R}^{p \times n}$, $\mathbf{C} \in \mathbb{R}^{m \times n}$                                         | $2mnp$      |

* Typical BLAS functions support single precision (S), double precision (D), complex (C), and double complex (Z). 

* Some operations _appear_ as level-3 but indeed are level-2.  
    - A common operation in statistics is column scaling or row scaling
    $$
    \begin{eqnarray*}
        \mathbf{A} &=& \mathbf{A} \mathbf{D} \quad \text{(column scaling)} \\
        \mathbf{A} &=& \mathbf{D} \mathbf{A} \quad \text{(row scaling)},
    \end{eqnarray*}
    $$
    where $\mathbf{D}$ is diagonal.  
    - These are essentially level-2 operations!

In [1]:
using BenchmarkTools

srand(123) # seed
n = 2000
A = rand(n, n)
d = rand(n)  # d vector
D = diagm(d) # diagonal matrix with d as diagonal

# this is calling BLAS routine for matrix multiplication: O(n^3) flops
@benchmark $A * $D

BenchmarkTools.Trial: 
  memory estimate:  30.52 MiB
  allocs estimate:  2
  --------------
  minimum time:     102.485 ms (2.38% GC)
  median time:      108.130 ms (2.32% GC)
  mean time:        127.424 ms (3.53% GC)
  maximum time:     242.041 ms (1.30% GC)
  --------------
  samples:          40
  evals/sample:     1

In [2]:
# columnwise scaling: O(n^2) flops
@benchmark scale($A, $d)

LoadError: [91mUndefVarError: scale not defined[39m

In [3]:
# current way for columnwise scaling: O(n^2) flops
@benchmark $A * Diagonal($d)

BenchmarkTools.Trial: 
  memory estimate:  30.52 MiB
  allocs estimate:  2
  --------------
  minimum time:     7.326 ms (5.23% GC)
  median time:      9.309 ms (24.99% GC)
  mean time:        9.600 ms (25.84% GC)
  maximum time:     65.366 ms (88.98% GC)
  --------------
  samples:          521
  evals/sample:     1

In [4]:
# in-place: avoid allocate space for result
@benchmark scale!($A, $d)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.484 ms (0.00% GC)
  median time:      11.481 ms (0.00% GC)
  mean time:        11.782 ms (0.00% GC)
  maximum time:     23.631 ms (0.00% GC)
  --------------
  samples:          424
  evals/sample:     1

## Memory hierarchy and level-3 fraction

> **Key to high performance is effective use of memory hierarchy. True on all architectures.**

* Flop count is not the sole determinant of algorithm efficiency. Another important factor is data movement through the memory hierarchy.

<img src="./macpro_inside.png" width="400" align="center">

<img src="http://hothardware.com/ContentImages/NewsItem/34743/content/Intel-Skylake-Graphics-Gen-9-Block-Diag.jpg" width="400" align="center">

<img src="http://images.bit-tech.net/content_images/2007/11/the_secrets_of_pc_memory_part_1/hei.png" width="400" align="center">

* Numbers everyone should know

| Operation                           | Time           |
|-------------------------------------|----------------|
| L1 cache reference                  | 0.5 ns         |
| L2 cache reference                  | 7 ns           |
| Main memory reference               | 100 ns         |
| Read 1 MB sequentially from memory  | 250,000 ns     |
| Read 1 MB sequentially from SSD     | 1,000,000 ns   |  
| Read 1 MB sequentially from disk    | 20,000,000 ns  |


<!-- | Operation                           | Time           | -->
<!-- |-------------------------------------|----------------| -->
<!-- | L1 cache reference                  | 0.5 ns         | -->
<!-- | Branch mispredict                   | 5 ns           | -->
<!-- | L2 cache reference                  | 7 ns           | -->
<!-- | Mutex lock/unlock                   | 100 ns         | -->
<!-- | Main memory reference               | 100 ns         | -->
<!-- | Compress 1K bytes with Zippy        | 10,000 ns      | -->
<!-- | Send 2K bytes over 1 Gbps network   | 20,000 ns      | -->
<!-- | Read 1 MB sequentially from memory  | 250,000 ns     | -->
<!-- | Round trip within same datacenter   | 500,000 ns     | -->
<!-- | Disk seek                           | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from network | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from disk    | 30,000,000 ns  | -->
<!-- | Send packet CA->Netherlands->CA     | 150,000,000 ns | -->

   Source: <https://gist.github.com/jboner/2841832>  

* For example, Xeon X5650 CPU has a theoretical throughput of 128 DP GFLOPS but a max memory bandwidth of 32GB/s.  

* Can we keep CPU cores busy with enough deliveries of matrix data and ship the results to memory fast enough to avoid backlog?  
Answer: use **high-level BLAS** as much as possible.

| BLAS                                                           | Dimension                                                                           | Mem. Refs. | Flops  | Ratio |
|----------------------------------------------------------------|-------------------------------------------------------------------------------------|------------|--------|-------|
| Level 1: $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$     | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$                                           | $3n$       | $2n$   | 3:2   |
| Level 2: $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$, $\mathbf{A} \in \mathbb{R}^{n \times n}$ | $n^2$      | $2n^2$ | 1:2   |
| Level 3: $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$ | $\mathbf{A}, \mathbf{B}, \mathbf{C} \in\mathbb{R}^{n \times n}$                    | $4n^2$     | $2n^3$ | 2:n |  

* Higher level BLAS (3 or 2) make more effective use of arithmetic logic units (ALU) by keeping them busy. **Surface-to-volume** effect.  
See [Dongarra slides](https://www.samsi.info/wp-content/uploads/2017/02/SAMSI-0217_Dongarra.pdf).

<img src="./blas_throughput.png" width="500" align="center"/>

* A distinction between LAPACK and LINPACK (older version of R uses LINPACK) is that LAPACK makes use of higher level BLAS as much as possible (usually by smart partitioning) to increase the so-called **level-3 fraction**.

## Effect of data layout

* Data layout in memory affects algorithmic efficiency too. It is much faster to move chunks of data in memory than retrieving/writing scattered data.

* Storage mode: **column-major** (Fortran, Matlab, R, Julia) vs **row-major** (C/C++).

* **Cache line** is the minimum amount of cache which can be loaded and stored to memory.
    - x86 CPUs: 64 bytes  
    - ARM CPUS: 32 bytes

<img src="https://patterns.eecs.berkeley.edu/wordpress/wp-content/uploads/2013/04/dense02.png" width="500" align="center"/>

* Accessing column-major stored matrix by rows causes lots of **cache misses**.

* Take matrix multiplication as an example 
$$ 
\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}, \quad \mathbf{A} \in \mathbb{R}^{m \times p}, \mathbf{B} \in \mathbb{R}^{p \times n}, \mathbf{C} \in \mathbb{R}^{m \times n}.
$$
Assume the storage is column-major, such as in Julia. There are 6 variants of the algorithms according to the order in the triple loops. 
    - `jki` or `kji` looping:
        ```julia
        for i = 1:m
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ikj` or `kij` looping:
        ```julia
        for j = 1:n
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ijk` or `jik` looping:
        ```julia
        for k = 1:p
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```
* We pay attention to the innermost loop, where the vector calculation occurs. The associated **stride** when accessing the three matrices in memory (assuming column-major storage) is  

| Variant        | A Stride | B Stride | C Stride |
|----------------|----------|----------|----------|
| $jki$ or $kji$ | Unit     | 0        | Unit     |
| $ikj$ or $kij$ | 0        | Non-Unit | Non-Unit |
| $ijk$ or $jik$ | Non-Unit | Unit     | 0        |       
Apparently the variants $jki$ or $kji$ are preferred.

In [5]:
"""
    matmul_by_loop!(A, B, C, order)

Overwrite `C` by `A * B`. `order` indicates the looping order for triple loop.
"""
function matmul_by_loop!(A::Matrix, B::Matrix, C::Matrix, order::String)
    
    m = size(A, 1)
    p = size(A, 2)
    n = size(B, 2)
    
    if order == "jki"
        for j = 1:n, k = 1:p, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kji"
        for k = 1:p, j = 1:n, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ikj"
        for i = 1:m, k = 1:p, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kij"
        for k = 1:p, i = 1:m, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ijk"
        for i = 1:m, j = 1:n, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "jik"
        for j = 1:n, i = 1:m, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
end

srand(123)
m, n, p = 2000, 100, 2000
A = rand(m, n)
B = rand(n, p)
C = zeros(m, p);

* $jki$ and $kji$ looping:

In [6]:
using BenchmarkTools

fill!(C, 0.0)
@benchmark matmul_by_loop!($A, $B, $C, "jki")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     430.380 ms (0.00% GC)
  median time:      440.254 ms (0.00% GC)
  mean time:        441.995 ms (0.00% GC)
  maximum time:     461.509 ms (0.00% GC)
  --------------
  samples:          12
  evals/sample:     1

In [7]:
fill!(C, 0.0)
@benchmark matmul_by_loop!($A, $B, $C, "kji")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     468.783 ms (0.00% GC)
  median time:      505.930 ms (0.00% GC)
  mean time:        499.414 ms (0.00% GC)
  maximum time:     521.797 ms (0.00% GC)
  --------------
  samples:          11
  evals/sample:     1

* $ikj$ and $kij$ looping:

In [8]:
fill!(C, 0.0)
@benchmark matmul_by_loop!($A, $B, $C, "ikj")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.306 s (0.00% GC)
  median time:      2.315 s (0.00% GC)
  mean time:        2.315 s (0.00% GC)
  maximum time:     2.323 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

In [9]:
fill!(C, 0.0)
@benchmark matmul_by_loop!($A, $B, $C, "kij")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.450 s (0.00% GC)
  median time:      2.532 s (0.00% GC)
  mean time:        2.532 s (0.00% GC)
  maximum time:     2.614 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1

* $ijk$ and $jik$ looping:

In [10]:
fill!(C, 0.0)
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     946.667 ms (0.00% GC)
  median time:      981.368 ms (0.00% GC)
  mean time:        1.033 s (0.00% GC)
  maximum time:     1.238 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

In [11]:
fill!(C, 0.0)
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     939.194 ms (0.00% GC)
  median time:      956.918 ms (0.00% GC)
  mean time:        983.601 ms (0.00% GC)
  maximum time:     1.050 s (0.00% GC)
  --------------
  samples:          6
  evals/sample:     1

* Julia wrapper of BLAS function. We see BLAS library wins hands down (multi-threading, Strassen algorithm, higher level-3 fraction by block outer product).

In [12]:
fill!(C, 0.0)
@benchmark Base.LinAlg.BLAS.gemm!('N', 'N', 1.0, A, B, 1.0, C)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.825 ms (0.00% GC)
  median time:      5.550 ms (0.00% GC)
  mean time:        6.134 ms (0.00% GC)
  maximum time:     12.681 ms (0.00% GC)
  --------------
  samples:          813
  evals/sample:     1

To appreciate the efforts in an optimized BLAS implementation such as OpenBLAS (evolved from GotoBLAS), see the [Quora question](https://www.quora.com/What-algorithm-does-BLAS-use-for-matrix-multiplication-Of-all-the-considerations-e-g-cache-popular-instruction-sets-Big-O-etc-which-one-turned-out-to-be-the-primary-bottleneck), especially the [video](https://youtu.be/JzNpKDW07rw). Bottomline is 

> Make friends with (good implementations of) BLAS/LAPACK and use them as much as possible.

## Avoid memory allocation: some examples

* Transposing matrix is an expensive memory operation.  
    - In R, the command 
        ```R
        t(A) %*% x
        ```
    will first transpose `A` then perform matrix multiplication, causing unnecessary memory allocation
    - Julia is smart to avoid transposing matrix if possible.

In [13]:
srand(123)

n = 1000
A = rand(n, n)
x = rand(n)

# dispatch to At_mul_B (and then to BLAS)
# does *not* actually transpose the matrix
@benchmark A' * x

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     87.706 μs (0.00% GC)
  median time:      122.672 μs (0.00% GC)
  mean time:        161.443 μs (0.00% GC)
  maximum time:     1.596 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [14]:
# dispatch to BLAS
@benchmark At_mul_B(A, x)

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     87.391 μs (0.00% GC)
  median time:      106.454 μs (0.00% GC)
  mean time:        115.580 μs (0.00% GC)
  maximum time:     588.206 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [15]:
# let's force transpose
@benchmark transpose(A) * x

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  3
  --------------
  minimum time:     2.873 ms (0.00% GC)
  median time:      3.737 ms (0.00% GC)
  mean time:        3.949 ms (15.79% GC)
  maximum time:     10.754 ms (27.78% GC)
  --------------
  samples:          1260
  evals/sample:     1

* [Broadcasting](https://docs.julialang.org/en/stable/manual/functions/#dot-syntax-for-vectorizing-functions) in Julia achieves vectorized code without creating intermediate arrays.

In [16]:
srand(123)
X, Y = rand(1000,1000), rand(1000,1000)

# two temporary arrays are created
@benchmark max(abs(X), abs(Y))

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,2}) at ./deprecated.jl:57
 [3] ##core#745() at /Users/huazhou/.julia/v0.6/BenchmarkTools/src/execution.jl:316
 [4] ##sample#746(::BenchmarkTools.Parameters) at /Users/huazhou/.julia/v0.6/BenchmarkTools/src/execution.jl:322
 [5] #_run#15(::Bool, ::String, ::Array{Any,1}, ::Function, ::BenchmarkTools.Benchmark{Symbol("##benchmark#744")}, ::BenchmarkTools.Parameters) at /Users/huazhou/.julia/v0.6/BenchmarkTools/src/execution.jl:350
 [6] (::BenchmarkTools.#kw##_run)(::Array{Any,1}, ::BenchmarkTools.#_run, ::BenchmarkTools.Benchmark{Symbol("##benchmark#744")}, ::BenchmarkTools.Parameters) at ./<missing>:0
 [7] anonymous at ./<missing>:?
 [8] #run_result#19(::Array{Any,1}, ::Function, ::BenchmarkTools.Benchmark{Symbol("##benchmark#744")}, ::BenchmarkTools.Parameters) at /Users/huazhou/.julia/v0.6/BenchmarkTools/src/execution.jl:44
 [9] (::BenchmarkTools.#kw##run_result)(::Array{Any,1}, ::BenchmarkTool

BenchmarkTools.Trial: 
  memory estimate:  22.92 MiB
  allocs estimate:  240
  --------------
  minimum time:     6.431 ms (17.05% GC)
  median time:      7.534 ms (17.45% GC)
  mean time:        9.039 ms (16.82% GC)
  maximum time:     25.927 ms (10.88% GC)
  --------------
  samples:          553
  evals/sample:     1

In [17]:
# no temporary arrays created
@benchmark max.(abs.(X), abs.(Y))

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  27
  --------------
  minimum time:     2.411 ms (0.00% GC)
  median time:      2.965 ms (0.00% GC)
  mean time:        3.539 ms (16.14% GC)
  maximum time:     10.373 ms (31.12% GC)
  --------------
  samples:          1410
  evals/sample:     1

In [18]:
# no memory allocation at all!
Z = zeros(X)
@benchmark Z .= max.(abs.(X), abs.(Y))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  4
  --------------
  minimum time:     1.894 ms (0.00% GC)
  median time:      2.023 ms (0.00% GC)
  mean time:        2.121 ms (0.00% GC)
  maximum time:     6.563 ms (0.00% GC)
  --------------
  samples:          2349
  evals/sample:     1

* [View](https://docs.julialang.org/en/stable/stdlib/arrays/#Base.view) avoids creating extra copy of matrix data.

In [19]:
srand(123)
A = randn(1000, 1000)

# sum entries in a sub-matrix
@benchmark sum(A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 
  memory estimate:  488.47 KiB
  allocs estimate:  5
  --------------
  minimum time:     63.184 μs (0.00% GC)
  median time:      258.860 μs (0.00% GC)
  mean time:        233.019 μs (11.53% GC)
  maximum time:     2.528 ms (79.41% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [20]:
# view avoids creating a separate sub-matrix
@benchmark sum(@view A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 
  memory estimate:  1.20 KiB
  allocs estimate:  36
  --------------
  minimum time:     98.066 μs (0.00% GC)
  median time:      106.817 μs (0.00% GC)
  mean time:        127.284 μs (0.00% GC)
  maximum time:     5.050 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

Julia 0.6 adds the [`@views`](https://docs.julialang.org/en/latest/manual/performance-tips.html#Consider-using-views-for-slices-1) macro, which can be useful in [some operations](https://discourse.julialang.org/t/why-is-a-manual-in-place-addition-so-much-faster-than-and-on-range-indexed-arrays/3302).